<a href="https://colab.research.google.com/github/theanh828726/ai-the-anh-multiapp/blob/main/Copy_of_SDVN_Musubi_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [![](https://img.shields.io/badge/Musubi-Tuner-Blue)](https://github.com/kohya-ss/musubi-tuner) [![](https://img.shields.io/badge/Video-Hướng%20dẫn-ff0000)](https://youtu.be/CHUAkrPhoig) [![](https://img.shields.io/badge/Design-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Ver-25.2308-0075ff)](https://trainlora.vn) [![](https://img.shields.io/badge/All%20Tools-trainlora.vn-0075ff)](https://trainlora.vn) [![](https://img.shields.io/badge/SDVN-Library-green)](https://bit.ly/sdvn-lib) [![](https://img.shields.io/badge/Khoá%20học-All%20in%20one-red)](https://hungdiffusion.com/) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG)
---
`✨ Support training lora Kontext, Qwen, Qwen-Edit`

#⚙️ I. Cài đặt

In [ ]:
# @title ###Install
import os
from google.colab import drive

drive.mount('/content/drive')
root_dir = "/content/"
data_dir = "/content/drive/MyDrive/SD-Data"
repo_dir = "/content/SDVN-musubi-tuner-colab"

if not os.path.exists("/content/musubi-tuner"):
  %cd /content
  !git clone https://github.com/StableDiffusionVN/SDVN-musubi-tuner-colab
  !git clone https://github.com/kohya-ss/musubi-tuner
  %cd /content/musubi-tuner
  !pip install -q -e .
  !pip install -q toml aria2 OpenAI google-genai flash_attn

%cd /content
%run {repo_dir}/TrainScript.ipynb

# 🎚️ II. Train Image (Qwen, Kontext)

In [ ]:
#@title 📂 2. Kết nối - xử lý data

ImageFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
# @markdown 🟡 `Thư mục control chỉ dùng cho train Kontext, Qwen-Edit`

# @markdown 🟡 `Đặt tên, vị trí thư mục như ImageFolder, đặt hậu tố _{số} (control_0.jpg) nếu muốn train nhiều control cho một image`
ControlFolder = ""  # @param {type:'string'}
DataClean = False #@param {type:"boolean"}
No_gen_caption = False #@param {type:"boolean"}
# @markdown 🟡 `OCR Prompt`

#@markdown 💡 ` Chạy Florence với Caption_Tools để tránh lỗi vặt, nên dùng Gemini API thay thế`

#@markdown [![](https://img.shields.io/badge/Caption-Tools-blue)](https://colab.research.google.com/github/StableDiffusionVN/SDVN-training-colab-flux/blob/main/SDVN_Caption_Tools.ipynb)

Caption_Image = 'None' # @param ['None', 'Florence', 'APIGemini | 2.5 Flash', ' APIGemini | 2.5 Pro', 'APIGemini | 2.5 Flash Lite',  'APIOpenAI | GPT 5', 'APIOpenAI | GPT 5-mini', 'APIOpenAI | GPT 5-nano']
Caption_Length = "Medium" # @param ["Short","Medium","Long"]
# @markdown 🟡 `API Caption`

# @markdown [![](https://img.shields.io/badge/Gemini-API-blue)](https://aistudio.google.com/app/apikey)
APIkey = "" # @param {type:'string'}
API_Prompt = "" # @param {type:'string'}
# @markdown 🟡 `Thêm caption tuỳ chọn`
Custom_Caption = "" # @param {type:'string'}
AddFolderName = False #@param {type:"boolean"}
Remove_Caption = False #@param {type:"boolean"}
Append = False #@param {type:"boolean"}

if No_gen_caption == True:
  Caption_Image = 'None'
  Caption_Video = 'None'
  Custom_Caption = ''
  AddFolderName = False

Cap_prompt = {
    'Short':['<CAPTION>',0.85, 30],
    'Medium':['<DETAILED_CAPTION>',0.5, 60],
    'Long':['<MORE_DETAILED_CAPTION>',0.35, 100]
}
extension = ".txt"

if DataClean == True :
  %cd /content
  clean_directory(ImageFolder)
if Caption_Image != "None":
  caption_dir(ImageFolder,Cap_prompt[Caption_Length])

list_train_dir = []
list_train_dir.append(ImageFolder)

if AddFolderName:
  for dir in list_train_dir:
    add_forder_name(dir)
if Custom_Caption != "":
  for dir in list_train_dir:
    process_dir(dir, Custom_Caption, Append, Remove_Caption)


if ControlFolder != "":
  import shutil
  shutil.copytree(ControlFolder, f"/content/control_folder/{ControlFolder.split('/')[-2]}", dirs_exist_ok=True)
  ControlFolder = f"/content/control_folder/{ControlFolder.split('/')[-2]}"
  config_control_image(ImageFolder,ControlFolder)


In [ ]:
#@title 🛠️ 3. Cấu hình train

%run {repo_dir}/Config.ipynb

model_type = "kontext_dev" # @param ["kontext_dev", "qwen_image", "qwen_image_edit"]
# @markdown 💡 `Model tuỳ chọn (Nhận link tải civitai, huggingface, drive path)`
model_train = "" # @param {type:'string'}
output_dir = "/content/drive/MyDrive/SD-Data/Lora" # @param {type:'string'}
output_name = "lora_name" # @param {type:'string'}
resolution = "1024" # @param {type:'string'}
batch_size = 1 # @param {type:'number'}
resize_control = True #@param {type:"boolean"}
image_dir = ImageFolder
num_repeats = 50 # @param {type:'number'}
max_train_epochs = 4 # @param {type:'number'}
save_every_n_epochs = 1 # @param {type:'number'}
save_last_n_epochs = 0 # @param {type:'number'}
save_every_n_steps = 0 # @param {type:'number'}

# @markdown *️⃣ `Điều chỉnh Learning Rate`
learning_rate = 1e-4 # @param {type:'number'}
optimizer_type = "adamw8bit" # @param ["adamw", "adamw8bit", "adafactor"]
lr_scheduler = "constant" # @param [ "cosine", "constant", "polynomial"]
network_dim = 64 # @param {type:'number'}
network_alpha = 32 # @param {type:'number'}
# @markdown *️⃣ `Chọn blocks to swap để giảm VRAM khi train Qwen Image, nện chọn với Qwen Edit`
blocks_to_swap = False #@param {type:"boolean"}

# @markdown *️⃣ `Bật/Tắt các tính năng mở rộng, bao gồm (info, sampler, sao lưu, log)`
extra_option = True #@param {type:"boolean"}
author = '' # @param {"type":"string"}
description = '' # @param {"type":"string"}
sample_prompt = "" #@param {type:'string'}
sample_every_n_steps = 100 # @param {type:'number'}
# @markdown 🟡 `Train kontext sẽ cần một ảnh đầu để chạy được sample, bỏ qua sẽ chọn tự động`
sample_image_path = "" #@param {type:'string'}
wandb_api_key = '' #@param {type:'string'}
save_state = False #@param {type:"boolean"}
save_state_on_train_end = False #@param {type:"boolean"}
resume = "" #@param {type:'string'}

#default
resolution = [int(x) for x in resolution.split(",")]
if len(resolution) == 1:
  resolution = [resolution[0],resolution[0]]
sample_size = resolution
dataset = os.path.join(root_dir,"dataset.toml")

data_config = data_config()
dataset_file()
config = config()
extra = extra()
folder_train = ImageFolder
prompt(sample_prompt, sample_image_path)
cache_latents_config = cache_latents_config()
cache_text_encoder_config = cache_text_encoder_config()

arg = f'accelerate launch {dic2arg(op)} {dic2arg(config)} {dic2arg(extra) if extra_option else ""}'
print(arg)
arg_latents_config = dic2arg(cache_latents_config)
print(arg_latents_config)
arg_text_encoder_config = dic2arg(cache_text_encoder_config)
print(arg_text_encoder_config)
print("Done")

In [ ]:
#@title ⏳ 3. Training
%cd /content/musubi-tuner
run_cache = True #@param {type:"boolean"}
if run_cache:
  !{arg_latents_config}
  !{arg_text_encoder_config}
!{arg}

AutoDisconect = False # @param {"type":"boolean"}
if AutoDisconect:
  from time import sleep
  sleep(3*60)
  from google.colab import runtime
  runtime.unassign()

# 🎚️ III. Train Video (Wan, Hun) - Coming soon